In [1]:
# Import libraries
import numpy as np
import pandas as pd
import seaborn as sns
from google.colab import drive
from sklearn import preprocessing
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from random import shuffle
from sklearn import svm
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import sys
import veritastool
import veritas

In [ ]:
from veritastool.model.modelwrapper import ModelWrapper
from veritastool.model.model_container import ModelContainer
from veritastool.usecases.predictive_underwriting import PredictiveUnderwriting

In [33]:
# Mount drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [34]:
df = pd.read_csv('gdrive/My Drive/Accenture x Turing/Data/Experiment_2/Rerun_6_pc/Clean_data_25_6_percent_bat_26_no_bot_85_linspace.csv')
df.drop(columns=df.columns[0], axis=1, inplace=True)
print(len(df))
df.head()

1111


,subtest_27,subtest_28,subtest_29,subtest_30,subtest_32,subtest_33,subtest_36,subtest_37,subtest_38,subtest_39,subtest_40,Model_C_label,Model_A_label,Model_B_label
0,1.000000,0.833333,0.733333,0.714286,0.798507,0.571429,1.000000,1.000000,0.985294,0.970930,0.978947,1,1,1
1,0.777778,0.833333,0.700000,0.428571,0.798507,1.000000,1.000000,1.000000,0.970588,0.976744,0.968421,1,1,0
2,0.777778,1.000000,0.700000,1.000000,0.729478,0.857143,0.727273,1.000000,1.000000,1.000000,0.963158,0,1,1
3,0.888889,0.833333,0.900000,0.571429,0.805970,0.571429,1.000000,0.909091,0.955882,0.976744,0.942105,1,1,1
4,1.000000,0.666667,0.966667,0.714286,0.751866,0.428571,1.000000,0.909091,0.985294,0.970930,0.963158,1,1,1


In [35]:
# Split labels and features

X_data, Y_data_C, Y_data_A, Y_data_B = df.iloc[:, :-3], df.iloc[:, [-3]], df.iloc[:, [-2]], df.iloc[:, [-1]]

# X_data, Y_data = df.iloc[:, :-1], df.iloc[:, [-1]]

In [36]:
Y_data_B = Y_data_B.reset_index()
Y_data_C = Y_data_C.reset_index()

In [37]:
# Split the dataset

X_train, X_test, y_train_A, y_test_A = train_test_split(X_data, Y_data_A, test_size=0.2)
y_train_A = y_train_A.reset_index()
y_test_A = y_test_A.reset_index()

In [38]:
y_train_B = pd.merge(y_train_A,Y_data_B[['index', 'Model_B_label']],on='index', how='left')

y_train_C = pd.merge(y_train_A,Y_data_C[['index', 'Model_C_label']],on='index', how='left')

y_test_B = pd.merge(y_test_A,Y_data_B[['index', 'Model_B_label']],on='index', how='left')

y_test_C = pd.merge(y_test_A,Y_data_C[['index', 'Model_C_label']],on='index', how='left')

# X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, test_size=0.2)
# print(len(X_train))
# print(len(X_test))
# print(len(y_train))
# print(len(y_test))

In [39]:
y_train_B = y_train_B.drop(labels='Model_A_label', axis = 1)
y_train_C = y_train_C.drop(labels='Model_A_label', axis = 1)
y_test_B = y_test_B.drop(labels='Model_A_label', axis = 1)
y_test_C = y_test_C.drop(labels='Model_A_label', axis = 1)

In [40]:
y_train_A = y_train_A.set_index("index")
y_train_B = y_train_B.set_index("index")
y_train_C = y_train_C.set_index("index")
y_test_A = y_test_A.set_index("index")
y_test_B = y_test_B.set_index("index")
y_test_C = y_test_C.set_index("index")

In [41]:
y_train_C.head()

,Model_C_label
index,
239,0
118,0
738,0
394,0
978,0


In [42]:
# Create a svm Classifier
clf = svm.SVC(kernel="poly", probability = True) # Polynomial Kernel

In [43]:
# Train the model using the training sets
model = clf.fit(X_train, y_train_A.values.ravel())

In [44]:
# Predict the response for test dataset
y_pred = clf.predict(X_test)
# print(y_pred)

In [45]:
# Predict the response for test dataset with probablities
y_pred_proba = clf.predict_proba(X_test)
array1 = y_pred_proba[:, 0].reshape(-1, 1).flatten()
array2 = y_pred_proba[:, 1].reshape(-1, 1).flatten()

In [46]:
from veritastool.model.modelwrapper import ModelWrapper
from veritastool.model.model_container import ModelContainer
from veritastool.usecases.predictive_underwriting import PredictiveUnderwriting

ModuleNotFoundError: ignored

In [ ]:
# Append predictions to X_test dataframe
X_test["Predicted"] = y_pred
X_test["Prob_0"] = array1
X_test["Prob_1"] = array2
X_train["Predicted"] = "train"
X_full = pd.concat([X_test, X_train])
X_full.to_csv('gdrive/My Drive/Accenture x Turing/Data/Test_X_A.csv')

In [ ]:
# Calculate accuracy
accuracy = metrics.accuracy_score(y_test, y_pred)

# Calculate precision
precision = metrics.precision_score(y_test, y_pred)

# Calculate recall
recall = metrics.recall_score(y_test, y_pred)
print(len(X_data))

In [ ]:
# Write a function to train an SVM
def train_and_predict(name, X_train, X_test, y_train, y_test, kernel='poly'):
    # Define X and Y data
    name=name

    # Create a svm Classifier
    clf = svm.SVC(kernel=kernel, probability = True) # Polynomial Kernel

    # Train the model using the training sets
    model = clf.fit(X_train, y_train.values.ravel())

    # Predict the response for test dataset
    y_pred = clf.predict(X_test)
    cm = confusion_matrix(y_pred, y_test)

    # Predict the probabilities for test dataset
    y_pred_proba = clf.predict_proba(X_test)

    # Change class probabilities into 2 flat numpy arrays
    array1 = y_pred_proba[:, 0].reshape(-1, 1).flatten()
    array2 = y_pred_proba[:, 1].reshape(-1, 1).flatten()

    # Append predictions to X_test dataframe
    X_eval = X_test.copy(deep=True)
    X_eval[f"Predicted_%s" % name] = y_pred

    # Append probability predictions to X_test dataframe
    X_eval[f"Prob_0_%s" % name] = array1
    X_eval[f"Prob_1_%s" % name] = array2

    # Mark which data was used for training
    X_tr = X_train.copy(deep = True)
    X_tr[f"Predicted_%s" % name] = "train"

    # Concatenate training and test data
    X_full = pd.concat([X_eval, X_tr])

    # Reset index and retain old index to be able to get back to sensitive data
    X_full = X_full.reset_index()

    # Calculate accuracy
    accuracy = metrics.accuracy_score(y_test, y_pred)

    # Calculate precision
    precision = metrics.precision_score(y_test, y_pred)

    # Calculate recall
    recall = metrics.recall_score(y_test, y_pred)

    return accuracy, precision, recall, X_full, cm

In [ ]:
# Call the function for each model
accuracy_A, precision_A, recall_A, X_full_A, cm_A = train_and_predict("A", X_train, X_test, y_train_A, y_test_A)
accuracy_B, precision_B, recall_B, X_full_B, cm_B = train_and_predict("B", X_train, X_test, y_train_B, y_test_B)
accuracy_C, precision_C, recall_C, X_full_C, cm_C = train_and_predict("C", X_train, X_test, y_train_C, y_test_C)

# Print the accuracy, precision, and recall for each model
print("Model A - Accuracy:", accuracy_A)
print("Model A - Precision:", precision_A)
print("Model A - Recall:", recall_A)

print("Model B - Accuracy:", accuracy_B)
print("Model B - Precision:", precision_B)
print("Model B - Recall:", recall_B)

print("Model C - Accuracy:", accuracy_C)
print("Model C - Precision:", precision_C)
print("Model C - Recall:", recall_C)

In [ ]:
# # Write a function to train an SVM
# def train_and_predict(df, name, test_size=0.2, kernel='poly'):
#     # Define X and Y data
#     df.drop(columns=df.columns[0], axis=1, inplace=True)
#     name=name
#     X_data, Y_data = df.iloc[:, :-1], df.iloc[:, [-1]]

#     # Split the dataset
#     X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, test_size=test_size)

#     # Create a svm Classifier
#     clf = svm.SVC(kernel=kernel, probability = True) # Polynomial Kernel

#     # Train the model using the training sets
#     model = clf.fit(X_train, y_train.values.ravel())

#     # Predict the response for test dataset
#     y_pred = clf.predict(X_test)
#     cm = confusion_matrix(y_pred, y_test)

#     # Predict the probabilities for test dataset
#     y_pred_proba = clf.predict_proba(X_test)

#     # Change class probabilities into 2 flat numpy arrays
#     array1 = y_pred_proba[:, 0].reshape(-1, 1).flatten()
#     array2 = y_pred_proba[:, 1].reshape(-1, 1).flatten()

#     # Append predictions to X_test dataframe
#     X_test[f"Predicted_%s" % name] = y_pred

#     # Append probability predictions to X_test dataframe
#     X_test[f"Prob_0_%s" % name] = array1
#     X_test[f"Prob_1_%s" % name] = array2

#     # Mark which data was used for training
#     X_train[f"Predicted_%s" % name] = "train"

#     # Concatenate training and test data
#     X_full = pd.concat([X_test, X_train])

#     # Reset index and retain old index to be able to get back to sensitive data
#     X_full = X_full.reset_index()

#     # Calculate accuracy
#     accuracy = metrics.accuracy_score(y_test, y_pred)

#     # Calculate precision
#     precision = metrics.precision_score(y_test, y_pred)

#     # Calculate recall
#     recall = metrics.recall_score(y_test, y_pred)

#     return accuracy, precision, recall, X_full, cm

In [ ]:
# # Call the function for each dataframe
# A = pd.read_csv('gdrive/My Drive/Accenture x Turing/Data/Experiment_2/Model_A_25_625_percent_bat_26_prob_no_bottom_85th_linspace.csv')
# accuracy_A, precision_A, recall_A, X_full_A, cm_A = train_and_predict(A, "A")

# B = pd.read_csv('gdrive/My Drive/Accenture x Turing/Data/Experiment_2/Model_B_25_625_percent_bat_26_prob_no_bottom_85th_linspace.csv')
# accuracy_B, precision_B, recall_B, X_full_B, cm_B = train_and_predict(B, "B")

# C = pd.read_csv('gdrive/My Drive/Accenture x Turing/Data/Experiment_2/Model_C_25_625_percent_bat_26_prob_no_bottom_85th_linspace.csv')
# accuracy_C, precision_C, recall_C, X_full_C, cm_C = train_and_predict(C, "C")

# # Print the accuracy, precision, and recall for each model
# print("Model A - Accuracy:", accuracy_A)
# print("Model A - Precision:", precision_A)
# print("Model A - Recall:", recall_A)

# print("Model B - Accuracy:", accuracy_B)
# print("Model B - Precision:", precision_B)
# print("Model B - Recall:", recall_B)

# print("Model C - Accuracy:", accuracy_C)
# print("Model C - Precision:", precision_C)
# print("Model C - Recall:", recall_C)

In [ ]:
X_full_A.head()

In [ ]:
# Merge dataframes for each model on old index; retain subtest results and all predictions for 3 models
full = pd.merge(X_full_A,X_full_B[['index','Predicted_B', 'Prob_0_B', "Prob_1_B"]],on='index', how='left')
full = pd.merge(full,X_full_C[['index','Predicted_C', 'Prob_0_C', "Prob_1_C"]],on='index', how='left')
full.head()

In [ ]:
# Look at the original data to get protected characteristics

orig_df = pd.read_csv('gdrive/My Drive/Accenture x Turing/Data/Experiment_2/Rerun_6_pc/All_models_25_6_percent_bat_26_prob_no_bottom_85th_linspace.csv')
orig_df = orig_df.rename(columns={'Unnamed: 0': 'index'})
orig_df.head()

In [ ]:
complete = pd.merge(full,orig_df[['index', 'age', 'gender', 'education_level', 'country', 'Model_A_label', 'Model_B_label', 'Model_C_label']],on='index', how='left')
labels_to_drop = ["subtest_27", "subtest_28", "subtest_29", "subtest_30", "subtest_32", "subtest_33", "subtest_36", "subtest_37", "subtest_38", "subtest_39", "subtest_40"]
complete = complete.drop(labels_to_drop, axis = 1)
complete.to_csv('gdrive/My Drive/Accenture x Turing/Data/Experiment_2/Rerun_6_pc/Complete_results_exp_2.csv')
cm_A = pd.DataFrame(cm_A)
cm_B = pd.DataFrame(cm_B)
cm_C = pd.DataFrame(cm_C)
cm_A.to_csv('gdrive/My Drive/Accenture x Turing/Data/Experiment_2/Rerun_6_pc/cm_A.csv')
cm_B.to_csv('gdrive/My Drive/Accenture x Turing/Data/Experiment_2/Rerun_6_pc/cm_B.csv')
cm_C.to_csv('gdrive/My Drive/Accenture x Turing/Data/Experiment_2/Rerun_6_pc/cm_C.csv')

In [ ]:
# Plot confusion matrices

# create heatmap of confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm_B, annot=True, cmap='Blues', fmt='g')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
# Select train data
train = df.loc[df["Predicted_A"] != "train"]

# Visualise test data properties - Education
labels_education = train['education_level'].unique()
education_dict = {
    1: 'Some high school',
    2: 'High school diploma / GED',
    3: 'Some college',
    4: 'College degree',
    5: 'Professional degree',
    6: "Master's degree",
    7: 'Ph.D.',
    8: "Associate's degree",
    99: 'Other'
}

# Replace the numbers with the corresponding education level strings
new_labels_education = list(map(lambda x: education_dict.get(int(x), 'Unknown'), labels_education))
train=train.replace({"education_level": education_dict})
counts_education = train['education_level'].value_counts()
counts_education = counts_education.to_dict()
types = list(counts_education.keys())
sizes = list(counts_education.values())
# print(counts_education)
# sizes_education = counts_education

# Visualise test data properties - Gender
labels_gender = train['gender'].unique()
no_f, no_m = train['gender'].value_counts()
sizes_gender = [no_f, no_m]

# Visualise test data properties - Age
labels_age = train['age'].unique()
ages = train['age'].value_counts()

# Create the histogram
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12,4))

# Plot histogram of ages in the first subplot
ax1.hist(ages.index, weights=ages.values, bins=range(18, 81, 5), edgecolor='black')
ax1.set_xlabel('Age')
ax1.set_ylabel('Frequency')
ax1.set_title('Age Distribution')

# Plot pie chart for Gender in the second subplot
ax2.pie(sizes_gender, labels=labels_gender, autopct='%1.0f%%')
ax2.set_title('Gender')

# Plot pie chart for Education in the third subplot
ax3.pie(sizes, labels=types, autopct='%1.0f%%')
ax3.set_title('Education')

# Show the plot
plt.show()

In [ ]:
Train_data = pd.merge(X_train, df_mod_A_orig, left_index=True, right_index=True)
Train_data_full = Train_data.iloc[:, 8:27]
Train_data_full = Train_data_full.iloc[:, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 18]]
Train_data_full.head()

In [ ]:
# Visualise train data properties - Education
labels_education_train = Train_data_full['education_level'].unique()
education_dict = {
    1: 'Some high school',
    2: 'High school diploma / GED',
    3: 'Some college',
    4: 'College degree',
    5: 'Professional degree',
    6: "Master's degree",
    7: 'Ph.D.',
    8: "Associate's degree",
    99: 'Other'
}

# Replace the numbers with the corresponding education level strings
new_labels_education_train = list(map(lambda x: education_dict.get(int(x), 'Unknown'), labels_education_train))
counts_education_train = Train_data_full['education_level'].value_counts()
sizes_education_train = counts_education_train

# Visualise test data properties - Gender
labels_gender_train = Train_data_full['gender'].unique()
no_f_train, no_m_train = Train_data_full['gender'].value_counts()
sizes_gender_train = [no_f_train, no_m_train]

# Visualise test data properties - Age
labels_age_train = Train_data_full['age'].unique()
ages_train = Train_data_full['age'].value_counts()

# Create the histogram
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12,4))

# Plot histogram of ages in the first subplot
ax1.hist(ages_train.index, weights=ages_train.values, bins=range(18, 81, 5), edgecolor='black')
ax1.set_xlabel('Age')
ax1.set_ylabel('Frequency')
ax1.set_title('Age Distribution')

# Plot pie chart for Gender in the second subplot
ax2.pie(sizes_gender_train, labels=labels_gender_train, autopct='%1.0f%%')
ax2.set_title('Gender')

# Plot pie chart for Education in the third subplot
ax3.pie(sizes_education_train, labels=new_labels_education_train, autopct='%1.0f%%')
ax3.set_title('Education')

# Show the plot
plt.show()

In [ ]:
# Select incorrectly classified data
True_positives = test_df_with_data[(test_df_with_data['Predicted'] == 1) & (test_df_with_data['Label_y'] == 1)]
True_negatives = test_df_with_data[(test_df_with_data['Predicted'] == 0) & (test_df_with_data['Label_y'] == 0)]
False_positives = test_df_with_data[(test_df_with_data['Predicted'] == 1) & (test_df_with_data['Label_y'] == 0)]
False_negatives = test_df_with_data[(test_df_with_data['Predicted'] == 0) & (test_df_with_data['Label_y'] == 1)]

# Visualise test data properties - Education
labels_education = True_positives['education_level'].unique()
education_dict = {
    1: 'Some high school',
    2: 'High school diploma / GED',
    3: 'Some college',
    4: 'College degree',
    5: 'Professional degree',
    6: "Master's degree",
    7: 'Ph.D.',
    8: "Associate's degree",
    99: 'Other'
}

# Replace the numbers with the corresponding education level strings
new_labels_education = list(map(lambda x: education_dict.get(int(x), 'Unknown'), labels_education))
counts_education = True_negatives['education_level'].value_counts()
sizes_education = counts_education

# Visualise test data properties - Gender
labels_gender = True_negatives['gender'].unique()
no_f, no_m = True_negatives['gender'].value_counts()
sizes_gender = [no_f, no_m]

# Visualise test data properties - Age
labels_age = True_negatives['age'].unique()
ages = True_negatives['age'].value_counts()

# Create the histogram
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12,4))

# Plot histogram of ages in the first subplot
ax1.hist(ages.index, weights=ages.values, bins=range(18, 81, 5), edgecolor='black')
ax1.set_xlabel('Age')
ax1.set_ylabel('Frequency')
ax1.set_title('Age Distribution')

# Plot pie chart for Gender in the second subplot
ax2.pie(sizes_gender, labels=labels_gender, autopct='%1.0f%%')
ax2.set_title('Gender')

# Plot pie chart for Education in the third subplot
ax3.pie(sizes_education, labels=new_labels_education, autopct='%1.0f%%')
ax3.set_title('Education')

# Show the plot
plt.show()

In [ ]:
# Extra code for additional measures if we want it
FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)  
FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
TP = np.diag(confusion_matrix)
TN = confusion_matrix.values.sum() - (FP + FN + TP)

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP) 
# Precision or positive predictive value
PPV = TP/(TP+FP)
# Negative predictive value
NPV = TN/(TN+FN)
# Fall out or false positive rate
FPR = FP/(FP+TN)
# False negative rate
FNR = FN/(TP+FN)
# False discovery rate
FDR = FP/(TP+FP)

# Overall accuracy
ACC = (TP+TN)/(TP+FP+FN+TN)